In [9]:
from tensorflow.keras.datasets import cifar10
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.applications.vgg19 import VGG19, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from keras_preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras import backend as BE
from tensorflow.keras.losses import categorical_crossentropy
from progressbar import ProgressBar
import os
import scipy.io
import tensorflow as tf
import json
import sys
sys.path.append('..')  
from GradPri_utils.utils import *

# from tensorflow.keras import backend as BE
# from Integrated_Gradients_algorithm import *
# from GradVisualizer import *

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
config=tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess=tf.compat.v1.Session(config=config) 

# tf.compat.v1.disable_eager_execution()
# tf.compat.v1.enable_eager_execution()

In [10]:
DATA_PATH = '../datasets/ImageNetVal/'
file_name = getfile_name(DATA_PATH)
file_name = np.sort(file_name)

f = open("../datasets/ILSVRC2012_devkit_t12/data/ILSVRC2012_validation_ground_truth.txt",encoding = "utf-8")
val_ground_truth = f.read()
val_ground_truth = val_ground_truth.split('\n')
for i in range(len(val_ground_truth)-1):
    val_ground_truth[i] = int(val_ground_truth[i])

    
vgg19_json = json.load(open('/public/liujiawei/.keras/models/imagenet_class_index.json','r',encoding="utf-8"))

In [11]:
base_model = VGG19(weights='imagenet')

In [12]:
synsets = scipy.io.loadmat(os.path.join('/public/liujiawei/huawei/ZHB/ADF-master/datasets', 'ILSVRC2012_devkit_t12', 'data', 'meta.mat'))['synsets']

ILSVRC2012_ID = [s[0][0][0][0] for s in synsets]

index1 = 821
WNID = [s[0][1][0] for s in synsets]
print(WNID[index1])

words = [s[0][2][0] for s in synsets]
print(words[index1])

num_train_images = [s[0][7][0][0] for s in synsets]
print(num_train_images[0])

n04263257
soup bowl
1300


In [13]:
MCP_p = []
top_set = 1 # 预测的前n个类中包含真实标签则表示预测正确
total_sample_num = 1000
img_num = range(0, total_sample_num)
ground_truth_label = []  # 记录样本的真实标签
predicted_confidence = []  # 记录样本的置信度信息

pbar = ProgressBar()
for i in pbar(img_num):
    img_path = DATA_PATH + file_name[i]
    img = image.load_img(img_path, target_size=(224, 224))
    x_tmp = image.img_to_array(img)
    x_tmp = np.expand_dims(x_tmp, axis=0)
    x_tmp = preprocess_input(x_tmp)
    pre_tmp = base_model.predict(np.reshape(x_tmp, [-1,224,224,3]))
    
    target_label = np.argsort(-pre_tmp) 
    
    MCP_p.append( pre_tmp[0][target_label[:,0][0]]/pre_tmp[0][target_label[:,1][0]]   )
    predicted_confidence.append(pre_tmp)
    ground_truth_label.append(WNID[val_ground_truth[i]-1])

100% |########################################################################|


In [21]:
indexs = np.argsort(MCP_p)
# indexs = indexs[::-1]
# 计算APFD指标
top_set=1
APFD,_,_ = get_APFD(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label), 
                    predicted_confidence=np.array(predicted_confidence), top_set=top_set,
                    decode_predictions=decode_predictions)
print("APFD: ", APFD)

# 计算RAUC指标
RAUC,_,_ = get_RAUC(Gini_indexs=indexs, ground_truth_label=np.array(ground_truth_label), 
                    predicted_confidence=np.array(predicted_confidence), top_set=top_set,
                    decode_predictions=decode_predictions)
print("RAUC: ", RAUC)

100% |########################################################################|


APFD:  0.7168219373219373
RAUC:  0.8682690149356816


In [8]:
print(pre_tmp[0][target_label[:,0][0]], pre_tmp[0][target_label[:,1][0]])

0.69561446 0.2315271
